![https://pieriantraining.com/](../PTCenteredPurple.png)

In this lecture we will use [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/lambda.html) to use the serverless computing service, [AWS Lambda](https://aws.amazon.com/lambda/)

## Hello World

Let's learn about aws lambda by creating a first *Hello World* application.

In [9]:
import boto3

### Create the Lambda function
It should simply return "Hello World" when invoked.
You have two possibilities to create the function:
1) Directly as a string here
2) Within a separate file that you load

99% of the time, you will have a separate python file that you load, so let's do this!

The main function is called *lambda_handler* and always accepts an **event** and **context**.<br />
Note that you can assign another name to this function as you pass it to the handler later

In [21]:
with open("hello.py", "r") as f:
    function_code = f.read()

In [22]:
print(function_code)

def lambda_handler(event, context):
    return "Hello World"


To create the lambda package you need to define:
1) The lambda function name
2) The runtime identifier (e.g python3.8) [List](https://docs.aws.amazon.com/lambda/latest/dg/lambda-runtimes.html)
3) The handler (name of your python function)
4) The IAM role

In [7]:
function_name = "HelloWorld1"
runtime = "python3.8"
handler = "lambda_function.lambda_handler"  # The first part is fixed, the second part is the name of your main python function

Regarding the IAM role, we can use the IAM client to create the corresponding role for this function

In [11]:
import json
# Create an IAM role for Lambda
iam_client = boto3.client('iam', region_name="us-east-1")

# Define the Lambda execution role policy
lambda_execution_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [  # Necessary for execution
                "logs:CreateLogGroup",  
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:*:*:*"
        }
    ]
}


role_name = 'LambdaExecutionRole'
role_response = iam_client.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=json.dumps({
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    })
)

policy_name = 'LambdaExecutionPolicy'
iam_client.put_role_policy(
    RoleName=role_name,
    PolicyName=policy_name,
    PolicyDocument=json.dumps(lambda_execution_policy)
)

# Get the ARN of the created role
role_arn = role_response['Role']['Arn']


In [12]:
role_arn

'arn:aws:iam::472948420345:role/LambdaExecutionRole'

Let's create the lambda function using client.create_function(*FunctionName*,*Runtime*,*Role*,*Handler*,*Code*) ([Doc](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/lambda/client/create_function.html))

You can upload the code via:
- 'ZipFile': b'bytes',
- 'S3Bucket': 'string',
- 'S3Key': 'string',
- 'S3ObjectVersion': 'string',
- 'ImageUri': 'string'

We can convert it to a zip file and use the file context to create the ZipFile

In [13]:
import zipfile
import io



In [14]:
lambda_client = boto3.client('lambda', region_name='us-east-1')

with io.BytesIO() as deployment_package:
    with zipfile.ZipFile(deployment_package, 'w') as zipf:
        zipf.writestr('lambda_function.py', function_code)

    create_function_response = lambda_client.create_function(
       FunctionName=function_name,
       Runtime=runtime,
       Role=role_arn,
       Handler=handler,
       Code={
           'ZipFile': deployment_package.getvalue()
       }
    )


Head over to lambda to check out your new function

## Invoke your function
To run your new lambda function, use client.invoke(*FunctionName*) [Doc](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/lambda/client/invoke.html)

In [15]:
invoke_response = lambda_client.invoke(FunctionName=function_name)

The result is stored in the *Payload* botocore and can be accessed using the read() method 

In [16]:
payload = invoke_response["Payload"].read()

In [17]:
payload.decode("utf-8")

'"Hello World"'

Congrats! You just created and ran your first aws lambda serverless function

## Delete Function
To delete your function, use client.delete_function(*FunctionName*)

In [18]:
lambda_client.delete_function(FunctionName=function_name)

{'ResponseMetadata': {'RequestId': '73a199d6-e843-4f2d-a332-dd56d76e29be',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'date': 'Mon, 11 Sep 2023 04:35:03 GMT',
   'content-type': 'application/json',
   'connection': 'keep-alive',
   'x-amzn-requestid': '73a199d6-e843-4f2d-a332-dd56d76e29be'},
  'RetryAttempts': 0}}

## Triggers

The power of aws lambda functions lies within its triggers.
Triggers allow us to invoke a lambda when a specific event occurs.

Let's create a new lambda function that prints the file name of a file uploaded to a specific s3 bucket

In [19]:
function_name = 'LambdaTrigger'
runtime = 'python3.8'
handler = 'lambda_function.lambda_trigger'  # Replace with your handler function


In [20]:
# Create an IAM role for Lambda trigger. Note that it needs to have s3 Read access
iam_client = boto3.client('iam', region_name="us-east-1")

# Define the Lambda execution role policy
lambda_execution_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:*:*:*"
        },
        {
            "Effect": "Allow",
            "Action": [  # Necessary to interact with s3 objects
                "s3:GetObject"
            ],
            "Resource": "arn:aws:s3:::*/*"
        }
    ]
}


role_name = 'LambdaExecutionRoleTrigger'
role_description = 'Role for Trigger Lambda'
role_response = iam_client.create_role(
    RoleName=role_name,
    Description=role_description,
    AssumeRolePolicyDocument=json.dumps({
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    })
)

policy_name = 'LambdaTriggerPolicy'
iam_client.put_role_policy(
    RoleName=role_name,
    PolicyName=policy_name,
    PolicyDocument=json.dumps(lambda_execution_policy)
)

# Get the ARN of the created role
role_arn = role_response['Role']['Arn']


The function code reads and prints the key of the uploaded object

In [23]:
with open("trigger.py", "r") as f:
    function_code = f.read()


In [24]:
print(function_code)

import json

def lambda_trigger(event, context):
    s3_object_key = event['Records'][0]['s3']['object']['key']
    print(f"File uploaded: {s3_object_key}")
    return s3_object_key


In [25]:
with io.BytesIO() as deployment_package:
    with zipfile.ZipFile(deployment_package, 'w') as zipf:
        zipf.writestr('lambda_function.py', function_code)

    create_function_response = lambda_client.create_function(
       FunctionName=function_name,
       Runtime=runtime,
       Role=role_arn,
       Handler=handler,
       Code={
           'ZipFile': deployment_package.getvalue()
       }
    )


### Setting up the S3 Trigger

In [26]:
s3_client = boto3.client('s3')
### Let's create a new bucket ###
bucket_name = "my-first-test-trigger"
s3_client.create_bucket(Bucket=bucket_name)


{'ResponseMetadata': {'RequestId': '0WNPC2EB8HYW2EKC',
  'HostId': 'BV2eIPQY3GTplHGt9doLEaLTwt6UVARGaS/03CK0OfkMJbLo91sU7XYmQtbK0q6FBkE9hdO6i2o=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'BV2eIPQY3GTplHGt9doLEaLTwt6UVARGaS/03CK0OfkMJbLo91sU7XYmQtbK0q6FBkE9hdO6i2o=',
   'x-amz-request-id': '0WNPC2EB8HYW2EKC',
   'date': 'Mon, 11 Sep 2023 04:36:22 GMT',
   'location': '/my-first-test-trigger',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': '/my-first-test-trigger'}

In [27]:
create_function_response["FunctionArn"]

'arn:aws:lambda:us-east-1:472948420345:function:LambdaTrigger'

In order to add the trigger we at first need to add the *InvokeFunction* permission to our lambda function.
To this end we can use client.add_permission(*FunctionName*,*StatementId*,*Action*,*Principal*,*SourceArn*) ([Doc](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/lambda/client/add_permission.html))

1) FunctionName : Name or ARN of your lambda
2) StatementId : Unique ID for this permission
3) Action : What action do you want to perform (i.e what permission do you want to change)
4) Principal : What AWS Service do you want to attach? (s3.amazonaws.com)
5) SourceArn : ARN of your s3 bucket

AddPermission needs to be manually added via IAM.  

## @ Jose - you probably need to show this in the video
## Add new inline permission -> lambda -> Either only add Permission or full access


```{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": "lambda:*",
            "Resource": "*"
        }
    ]
}

In [28]:
bucket_arn = "arn:aws:s3:::my-first-test-trigger"  # Change my-first-test-trigger to your bucket name
lambda_client.add_permission(
     FunctionName=function_name,
     StatementId='ID1',  # Unique statement ID
     Action='lambda:InvokeFunction',  # Allow to invoke the function
     Principal='s3.amazonaws.com',  # 
     SourceArn=bucket_arn,
 )


{'ResponseMetadata': {'RequestId': 'a801e760-e985-46eb-8e38-b6ab698c01ac',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Mon, 11 Sep 2023 04:36:26 GMT',
   'content-type': 'application/json',
   'content-length': '305',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'a801e760-e985-46eb-8e38-b6ab698c01ac'},
  'RetryAttempts': 0},
 'Statement': '{"Sid":"ID1","Effect":"Allow","Principal":{"Service":"s3.amazonaws.com"},"Action":"lambda:InvokeFunction","Resource":"arn:aws:lambda:us-east-1:472948420345:function:LambdaTrigger","Condition":{"ArnLike":{"AWS:SourceArn":"arn:aws:s3:::my-first-test-trigger"}}}'}

Next we add the trigger (notification_configuration) to our bucket using client.put_bucket_notification_configuration(*Bucket*, *NotificationConfiguration*) ([Doc](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3/client/put_bucket_notification_configuration.html))

The *NotificationConfiguration* is a LambdaFunctionConfigurations object that contains the LambdaFunctionArn and the actual [trigger event](https://docs.aws.amazon.com/AmazonS3/latest/API/API_LambdaFunctionConfiguration.html) (s3:ObjectCreated:*) 

In [29]:
# Define the event configuration
event_configuration = {
    'LambdaFunctionConfigurations': [
        {
            'LambdaFunctionArn': create_function_response["FunctionArn"],
            'Events': ['s3:ObjectCreated:*'],
        }
    ]
}

# Configure the S3 event trigger
s3_client.put_bucket_notification_configuration(
    Bucket=bucket_name,
    NotificationConfiguration=event_configuration
)



{'ResponseMetadata': {'RequestId': '80X07N82FZYEJGKP',
  'HostId': 'Ub4DM5YpxYl9g8tOJSUQQsU2v8D2FHjX08uLcwtHc4hufUrFZvdlzdGayLJgf11AZJz92j4Xh5Y=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Ub4DM5YpxYl9g8tOJSUQQsU2v8D2FHjX08uLcwtHc4hufUrFZvdlzdGayLJgf11AZJz92j4Xh5Y=',
   'x-amz-request-id': '80X07N82FZYEJGKP',
   'date': 'Mon, 11 Sep 2023 04:36:33 GMT',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0}}

If you check your lambda function, you should now see the S3 trigger

Let's upload a file:

In [30]:
with open("Test.txt", "w") as f:
    f.write("Hello World!")

In [31]:
s3_client.upload_file(Filename="Test.txt", Bucket=bucket_name, Key="Test_in_bucket.txt")

To inspect the log, we can interact with [Cloudwatch](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/cloudwatch.html)

In [32]:
cloudwatch_logs = boto3.client('logs', region_name="us-east-1")
response = cloudwatch_logs.describe_log_groups()

In [33]:
log_streams = cloudwatch_logs.describe_log_streams(
    logGroupName=f'/aws/lambda/{function_name}',
    orderBy='LastEventTime',
    descending=True
)["logStreams"]


In [34]:
cloudwatch_logs.get_log_events(
    logGroupName=f'/aws/lambda/{function_name}',
    logStreamName=log_streams[0]["logStreamName"]
)


{'events': [{'timestamp': 1694407001212,
   'message': 'INIT_START Runtime Version: python:3.8.v26\tRuntime Version ARN: arn:aws:lambda:us-east-1::runtime:81f077a44b32842209c33a8a4ecbe13f17dabc0964eaae632cb27846f04bf85e\n',
   'ingestionTime': 1694407005993},
  {'timestamp': 1694407001329,
   'message': 'START RequestId: 718a17e1-4ea9-498e-b65c-2cfb0c577155 Version: $LATEST\n',
   'ingestionTime': 1694407005993},
  {'timestamp': 1694407001329,
   'message': 'File uploaded: Test_in_bucket.txt\n',
   'ingestionTime': 1694407005993},
  {'timestamp': 1694407001330,
   'message': 'END RequestId: 718a17e1-4ea9-498e-b65c-2cfb0c577155\n',
   'ingestionTime': 1694407005993},
  {'timestamp': 1694407001331,
   'message': 'REPORT RequestId: 718a17e1-4ea9-498e-b65c-2cfb0c577155\tDuration: 1.69 ms\tBilled Duration: 2 ms\tMemory Size: 128 MB\tMax Memory Used: 38 MB\tInit Duration: 116.32 ms\t\n',
   'ingestionTime': 1694407005993}],
 'nextForwardToken': 'f/37786538799288969426672783571068587591312450

Last but not least, let's delete our function

In [35]:
lambda_client.delete_function(FunctionName=function_name)

{'ResponseMetadata': {'RequestId': 'adb2d6ca-4ef4-4f43-9a75-89db76503006',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'date': 'Mon, 11 Sep 2023 04:37:09 GMT',
   'content-type': 'application/json',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'adb2d6ca-4ef4-4f43-9a75-89db76503006'},
  'RetryAttempts': 0}}